In [24]:
import torch
import torch.nn as nn

from models.common import Conv, Concat, MP, SPPCSPC, RepConv
from models.yolo import IDetect
from utils.general import make_divisible
from copy import deepcopy


# IMPORT THIS!
from models.ELAN import BBoneELAN, HeadELAN

BBoneELAN

In [25]:
input = torch.randn(1, 128, 64, 64) #random input

# depth 2 -> 192
block = BBoneELAN(chan1=128, chan2=64, ker=3, depth=2)
print('Backbone EALN output shape (depth = 2) :', block(input).shape)

# depth 3 -> 256
block = BBoneELAN(chan1=128, chan2=64, ker=3, depth=3)
print('Backbone EALN output shape (depth = 3) :', block(input).shape)

# depth 4 -> 320
block = BBoneELAN(chan1=128, chan2=64, ker=3, depth=4)
print('Backbone EALN output shape (depth = 4) :', block(input).shape)

# depth 5 -> 384
block = BBoneELAN(chan1=128, chan2=64, ker=3, depth=5)
print('Backbone EALN output shape (depth = 5) :', block(input).shape)

Backbone EALN output shape (depth = 2) : torch.Size([1, 192, 64, 64])
Backbone EALN output shape (depth = 3) : torch.Size([1, 256, 64, 64])
Backbone EALN output shape (depth = 4) : torch.Size([1, 320, 64, 64])
Backbone EALN output shape (depth = 5) : torch.Size([1, 384, 64, 64])


Head ELAN

In [26]:
input = torch.randn(1, 512, 64, 64) #random input

# depth 2 -> 640
block = HeadELAN(chan1=512, chan2=256, ker=3, depth=2)
print('Head EALN output shape (depth = 2) :', block(input).shape)

# depth 3 -> 768
block = HeadELAN(chan1=512, chan2=256, ker=3, depth=3)
print('Head EALN output shape (depth = 3) :', block(input).shape)

# depth 4 -> 896
block = HeadELAN(chan1=512, chan2=256, ker=3, depth=4)
print('Head EALN output shape (depth = 4) :', block(input).shape)

# depth 5 -> 1024
block = HeadELAN(chan1=512, chan2=256, ker=3, depth=5)
print('Head EALN output shape (depth = 5) :', block(input).shape)

Head EALN output shape (depth = 2) : torch.Size([1, 640, 64, 64])
Head EALN output shape (depth = 3) : torch.Size([1, 768, 64, 64])
Head EALN output shape (depth = 4) : torch.Size([1, 896, 64, 64])
Head EALN output shape (depth = 5) : torch.Size([1, 1024, 64, 64])


YOLO Supernet

In [27]:
def parse_supernet(d, ch):  # model_dict, input_channels(3)
    print('\n%3s%18s%3s%10s  %-40s%-30s' % ('', 'from', 'n', 'params', 'module', 'arguments'))
    anchors, nc, gd, gw = d['anchors'], d['nc'], d['depth_multiple'], d['width_multiple']
    na = (len(anchors[0]) // 2) if isinstance(anchors, list) else anchors  # number of anchors
    no = na * (nc + 5)  # number of outputs = anchors * (classes + 5)

    layers, save, c2 = [], [], ch[-1]  # layers, savelist, ch out
    for i, (f, n, m, args) in enumerate(d['backbone'] + d['head']):  # from, number, module, args
        m = eval(m) if isinstance(m, str) else m  # eval strings
        for j, a in enumerate(args):
            try:
                args[j] = eval(a) if isinstance(a, str) else a  # eval strings
            except:
                pass

        n = max(round(n * gd), 1) if n > 1 else n  # depth gain
        if m in [nn.Conv2d, Conv, RepConv, SPPCSPC]:
            c1, c2 = ch[f], args[0]
            if c2 != no:  # if not output
                c2 = make_divisible(c2 * gw, 8)

            args = [c1, c2, *args[1:]]
            if m in [SPPCSPC]:
                args.insert(2, n)  # number of repeats
                n = 1
        elif m is BBoneELAN:
            c1, c2 = ch[f], int(args[0]*(args[-1]+1))
            args = [c1, *args]
        elif m is HeadELAN:
            c1, c2 = ch[f], int((args[0]*2) + (args[0]/2 * (args[-1]-1)))
            args = [c1, *args]
        elif m is nn.BatchNorm2d:
            args = [ch[f]]
        elif m is Concat:
            c2 = sum([ch[x] for x in f])
        elif m in [IDetect]:
            args.append([ch[x] for x in f])
            if isinstance(args[1], int):  # number of anchors
                args[1] = [list(range(args[1] * 2))] * len(f)

        else:
            c2 = ch[f]
        print(m)
        m_ = nn.Sequential(*[m(*args) for _ in range(n)]) if n > 1 else m(*args)  # module
        t = str(m)[8:-2].replace('__main__.', '')  # module type
        np = sum([x.numel() for x in m_.parameters()])  # number params
        m_.i, m_.f, m_.type, m_.np = i, f, t, np  # attach index, 'from' index, type, number params
        print('%3s%18s%3s%10.0f  %-40s%-30s' % (i, f, n, np, t, args))  # print
        save.extend(x % i for x in ([f] if isinstance(f, int) else f) if x != -1)  # append to savelist
        layers.append(m_)
        if i == 0:
            ch = []
        ch.append(c2)
    return nn.Sequential(*layers), sorted(save)

LOAD YAML

In [28]:
import yaml
yaml_file = '../yaml/yolov7_elan_test.yml'
with open(yaml_file) as f:
    yaml = yaml.load(f, Loader=yaml.SafeLoader)
    
# Define model
ch, nc, anchors = 3, None, None

ch = yaml['ch'] = yaml.get('ch', ch)  # input channels
if nc and nc != yaml['nc']:
    print(f"Overriding model.yaml nc={yaml['nc']} with nc={nc}")
    yaml['nc'] = nc  # override yaml value
if anchors:
    print(f'Overriding model.yaml anchors with anchors={anchors}')
    yaml['anchors'] = round(anchors)  # override yaml value

In [29]:
model, save = parse_supernet(deepcopy(yaml), ch=[ch])


                 from  n    params  module                                  arguments                     
<class 'models.common.Conv'>
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
<class 'models.common.Conv'>
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
<class 'models.common.Conv'>
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
<class 'models.common.Conv'>
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
<class 'models.ELAN.BBoneELAN'>
  4                -1  1    419840  models.ELAN.BBoneELAN                   [128, 64, 3, 3]               
<class 'models.common.Conv'>
  5                -1  1     66048  models.common.Conv                      [256, 256, 1, 1]              
<class 'models.common.MP'>
  6                -1  1         0  models.com

In [30]:
m = model[-1]

def forward_once(x, model, profile=False):
    y, dt = [], []  # outputs
    for m in model:
        if m.f != -1:  # if not from previous layer
            x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers

        x = m(x)  # run
        
        y.append(x if m.i in save else None)  # save output

    if profile:
        print('%.1fms total' % sum(dt))
    return x

input = torch.randn(1, 3, 256, 256)

forward_once(input, model)

[tensor([[[[[-4.75114e-01,  7.99749e-02, -8.90687e-02,  ...,  5.64646e-01, -1.48690e-01, -1.37017e-01],
            [-4.66355e-01, -4.38686e-02, -5.31136e-02,  ...,  1.95387e-01, -2.76595e-01, -6.63484e-02],
            [ 3.79253e-01, -1.95050e-01, -4.23512e-01,  ...,  2.48706e-01,  5.35180e-01, -3.98430e-02],
            ...,
            [-2.04867e-01, -1.30515e-01, -5.76746e-01,  ...,  5.24857e-01,  3.44864e-01, -5.71142e-02],
            [ 1.45665e-01, -2.19203e-01, -3.18572e-01,  ...,  6.83201e-02,  3.02671e-01, -3.28347e-01],
            [-3.08756e-01, -8.93085e-02, -6.01682e-02,  ...,  1.57259e-01,  5.80779e-01, -1.91867e-02]],
 
           [[ 1.54940e-01, -2.73262e-01, -4.72904e-01,  ...,  1.54893e-01, -4.76825e-02, -4.37791e-02],
            [-4.19791e-01, -1.17315e-01, -8.28878e-02,  ...,  1.22316e-02, -1.56766e-02,  3.61306e-01],
            [ 3.37567e-01,  5.48837e-01, -2.22683e-02,  ...,  6.09467e-01,  4.10375e-01,  1.48162e-02],
            ...,
            [-3.73618e-01, 